<a href="https://colab.research.google.com/github/MarcosACVieira/0_GERAL/blob/main/acessar_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
from time import sleep

In [ ]:
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1J0v42AlQ2Ba5WxkPzTL3uzdPiz_n9sBO'
download = drive.CreateFile({'id': file_id})
download.GetContentFile('file.csv')
cnpjs_target = pd.read_csv("file.csv")[['CNPJ_FUNDO','Banco']]

In [ ]:
df_all = pd.DataFrame()
months = 12
for i in range(months) :
    currentTimeDate = datetime.now() - relativedelta(months=i) #- timedelta(days=1)
    currentMonth = currentTimeDate.strftime('%Y%m')
    url = "https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_" + currentMonth + ".zip"
    print(url)
    df = pd.read_csv(url, compression='zip', sep = ';')
    df = df[df['CNPJ_FUNDO'].isin(cnpjs_target['CNPJ_FUNDO'])]
    df_all = pd.concat([df_all,df])

In [ ]:
# ibov, nasdaq, dowjones,ouro,bitcoin, cripto,Europa, China

In [ ]:
yfin = yf.download(tickers = ['^BVSP','^IXIC', '^DJI','GC=F','BTC-USD','^CMC200','FEZ','000001.SS'], period = '5y')
yfin = yfin[['Close']]
yfin.columns = yfin.columns.droplevel()
yfin['data'] = yfin.index
yfin = yfin.rename(columns={"^BVSP": "ibov", "^IXIC": "Nasdaq", "^DJI": "dowjones", "GC=F": "ouro", "BTC-USD": "Bitcoin", "^CMC200": "Criptos", "FEZ": "Europa", "000001.SS": "China"})

yfin

In [ ]:
url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json'
selic = pd.read_json(url)

url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.10844/dados?formato=json'
ipca = pd.read_json(url)

selic['data'] = pd.to_datetime(selic['data'], format='%d/%m/%Y')
ipca['data'] = pd.to_datetime(ipca['data'], format='%d/%m/%Y')
df_all['DT_COMPTC'] = pd.to_datetime(df_all['DT_COMPTC'])

# selic = selic[selic['data'].isin(df_all['DT_COMPTC'])]
# ipca = ipca[ipca['data'].isin(df_all['DT_COMPTC'])]

# df_all = df_all[df_all['DT_COMPTC'].isin(selic['data'])]
# df_all = df_all[df_all['DT_COMPTC'].isin(ipca['data'])]

# selic = selic[selic['data'].isin(df_all['DT_COMPTC'])]
# ipca = ipca[ipca['data'].isin(df_all['DT_COMPTC'])]

pivot = pd.pivot_table(df_all, values='VL_QUOTA', index='DT_COMPTC', columns='CNPJ_FUNDO')
selic = selic.set_index('data')
ipca = ipca.set_index('data')


In [ ]:
correl = pd.DataFrame()
correl['cnpj'] = list(pivot)

# acc =[]
# for col in range(pivot.shape[1]) :
#     pivot_col = pivot.iloc[:,col].dropna()
#     selic_copy = selic.loc[pivot_col.index]
#     acc = acc + [selic_copy.corrwith(pivot_col).iloc[0]]

# correl['selic'] = list(acc)

# acc =[]
# for col in range(pivot.shape[1]) :
#     pivot_col = pivot.iloc[:,col].dropna()
#     ipca_copy = ipca.loc[pivot_col.index]
#     acc = acc + [ipca_copy.corrwith(pivot_col).iloc[0]]

# correl['ipca'] = list(acc)

for k in range(len(list(yfin))-1) :
    asset = list(yfin)[k]
    acc =[]
    for col in range(pivot.shape[1]) :
        pivot_col = pivot.iloc[:,col].dropna()
        yfin_copy = yfin[[asset,'data']]
        yfin_copy = yfin_copy.loc[pivot_col.index]
        acc = acc + [yfin_copy.corrwith(pivot_col).iloc[0]]

    correl[asset] = list(acc)


In [ ]:
correl